7) Dado el dataset Genealogía el cual está formado por: 

<nombre_individuo, dni_individuo, dni_mamá>

realice distintas funciones que:



c) El nombre de la “abuela” que tiene más descendientes

d) Los nombres de los hermanos de la familia más numerosa (la cantidad de integrantes de una familia se calcula como la cantidad de hermanos más la mamá). Podría existir más de una familia más numerosa, en cuyo caso se deben imprimir todos los nombres de los hermanos integrantes de cada familia

In [4]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import col

sc = SparkContext("local", "My program")
sqlContext = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=My program, master=local) created by __init__ at <ipython-input-1-0b9e27f0d652>:5 

In [5]:
genealogia = sc.textFile("datasets/genealogia.txt")
genealogia = genealogia.map(lambda t : t.split("\t"))

genealogia = genealogia.map(lambda t: 
                               Row(
                                   nombre_individuo = str(t[0]),
                                   dni_individuo = int(t[1]),
                                   dni_mama = int(t[2]) if t[2] != 'None' else None 
                               )
                           )

genealogiaDF = sqlContext.createDataFrame(genealogia)
genealogiaDF.registerTempTable("Genealogia")

### a) Dado los dni de dos individuos indicar si son primos (dos individuos son primos si tienen la misma abuela).

In [6]:
dni1 = 646
dni2 = 223

def getAbuela(dni):
    query = f"""
    SELECT g.nombre_individuo, g.dni_individuo, abuela.nombre_individuo as nombre_abuela, abuela.dni_individuo as dni_abuela
    FROM Genealogia g
    INNER JOIN Genealogia madre ON g.dni_mama = madre.dni_individuo
    INNER JOIN Genealogia abuela ON madre.dni_mama = abuela.dni_individuo
    WHERE g.dni_individuo = '{ dni }';"""
    result = sqlContext.sql(query)
    return(result.select('dni_abuela').first()[0])

son_primos = getAbuela(dni1) == getAbuela(dni2)
print(f"{dni1} es primo de {dni2}: " + str(son_primos))

646 es primo de 223: True


### b) Dado los dni de dos individuos i1 y i2 indicar si i1 es ancestro de i2.

In [7]:
query = f"""
SELECT g.nombre_individuo, g.dni_individuo, abuela.nombre_individuo as nombre_abuela, abuela.dni_individuo as dni_abuela
FROM Genealogia g
INNER JOIN Genealogia madre ON g.dni_mama = madre.dni_individuo
INNER JOIN Genealogia abuela ON madre.dni_mama = abuela.dni_individuo"""

dni1 = 2184
dni2 = 646

def getMadre(id):
    query = f"""
    SELECT g.dni_mama
    FROM Genealogia g
    WHERE g.dni_individuo = '{ id }';"""
    result1 = sqlContext.sql(query)
    return result1.first()[0]


def getAncestros(id):
    ancestros = []
    madre = id
    while madre is not None:
        madre = getMadre(madre)
        if madre is not None:
            ancestros.append(madre)
    return ancestros


ancestros = getAncestros(dni2)
print(f"{dni1} es ancestro de {dni2}: " + str(dni1 in ancestros))

2184 es ancestro de 646: True


### c) El nombre de la “abuela” que tiene más descendientes

In [9]:
def getNietos():
    query = f"""
    SELECT abuela.nombre_individuo, abuela.dni_individuo, COUNT(*) as count
    FROM Genealogia g
    INNER JOIN Genealogia madre ON g.dni_mama = madre.dni_individuo 
    INNER JOIN Genealogia abuela ON abuela.dni_individuo = madre.dni_mama
    GROUP BY abuela.nombre_individuo, abuela.dni_individuo
    ORDER BY count DESC;"""
    result = sqlContext.sql(query)
    max = result.first()[2]
    print(max)
    result = result.filter(col("count") == max)
    return result.show()

getNietos()

9
+----------------+-------------+-----+
|nombre_individuo|dni_individuo|count|
+----------------+-------------+-----+
|         Czvqfzh|         1046|    9|
|          Tsdzfx|          960|    9|
|        Darpfeqd|         2785|    9|
|         Symqeqf|         4460|    9|
|          Khqrpf|          572|    9|
+----------------+-------------+-----+



### d) Los nombres de los hermanos de la familia más numerosa (la cantidad de integrantes de una familia se calcula como la cantidad de hermanos más la mamá). 
Podría existir más de una familia más numerosa, en cuyo caso se deben imprimir todos los nombres de los hermanos integrantes de cada familia

In [10]:
def getMadres():
    query = f"""
    SELECT g.dni_mama, COUNT(*) as count
    FROM Genealogia g
    WHERE g.dni_mama is not null
    GROUP BY g.dni_mama 
    ORDER BY count DESC"""
    result = sqlContext.sql(query)
    return result

madres = getMadres()
max = madres.first()[1]
madres = madres.filter(col("count") == max)

nombres = genealogiaDF.join(madres, madres.dni_mama == genealogiaDF.dni_mama).select("nombre_individuo", "g.dni_mama")
print(nombres.show())

+----------------+--------+
|nombre_individuo|dni_mama|
+----------------+--------+
|            pepo|     474|
|        Darpfeqd|     474|
|         Sqvbmee|     474|
|           Dybre|     474|
+----------------+--------+

None
